In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity


plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "text.usetex": False,  # Turn this OFF to prevent LaTeX errors
    "font.family": "serif",
    "pgf.rcfonts": False,

    "axes.labelsize": 14,  # Increase label font size
    "axes.titlesize": 16,  # Increase title font size
    "legend.fontsize": 12,  # Increase legend font size
    "xtick.labelsize": 15,  # Increase x-axis tick size
    "ytick.labelsize": 15,  # Increase y-axis tick size
    "lines.linewidth": 2.5,  # Make lines thicker
    "lines.markersize": 8,  # Increase marker size
    "grid.alpha": 0.3,  # Make grid less prominent
    "figure.figsize": (4, 3),  # Set consistent figure size
})


In [ ]:
heisenberg_valid_file = 'data/heisenberg_xyz_valid.h5'
ti_valid_file = 'data/ti_valid.h5'
fh_valid_file = 'data/fh_valid.h5'

molecule_valid_file = 'data/h2_valid.h5'

vqe_valid_file = 'data/random_vqe_valid.h5'


In [3]:
def plot_cosine_sim(valid_file, model_name='Heisenberg XYZ'):
    opt_vec = []
    diff_vec = []
    q_vec = []
    with h5py.File(valid_file,'r') as f:

        opt_param_group = f['model_param']
        diff_param_group = f['diff_model_param']
        q_param_group = f['dgnn_model_param']
        keys = list(opt_param_group.keys())
        
        for key in keys:
            opt_param = opt_param_group[key][...]
            opt_param = opt_param.flatten()
            diff_param = diff_param_group[key][...]
            diff_param = diff_param.flatten()
            q_param = q_param_group[key][...]
            q_param = q_param.flatten()
            opt_vec.append(opt_param)
            diff_vec.append(diff_param)
            q_vec.append(q_param)
        
    n_samples = len(opt_vec)

    sim_opt_diff = [cosine_similarity(opt_vec[i].reshape(1, -1), diff_vec[i].reshape(1, -1))[0,0] for i in range(n_samples)]
    sim_opt_q = [cosine_similarity(opt_vec[i].reshape(1, -1), q_vec[i].reshape(1, -1))[0,0] for i in range(n_samples)]


    sim_opt_diff = np.array(sim_opt_diff)
    sim_opt_q = np.array(sim_opt_q)

    sim_o_d_w = np.ones_like(sim_opt_diff) / (len(sim_opt_diff))

    sim_o_q_w = np.ones_like(sim_opt_q) / (len(sim_opt_q))


    bins = 80 

    hist_1, _ = np.histogram(sim_opt_diff, bins=bins, weights=sim_o_d_w)
    hist_2, _ = np.histogram(sim_opt_q, bins=bins, weights=sim_o_q_w)

    max_val = max(np.max(hist_1), np.max(hist_2))
    plt.figure()

    colors = ['limegreen', 'lightcoral']

    names = ['Diffusion', 'GNN']

    plt.hist([sim_opt_diff, sim_opt_q], bins=80, weights=[sim_o_d_w, sim_o_q_w], alpha=0.9, color=colors, label=names)
    
    y_top = max_val * 1.1

    yticks = np.linspace(0, y_top, 6)  # Define Y-ticks explicitly

    plt.yticks(yticks,labels=[f"{y:.2f}" for y in yticks] )
    plt.legend()
    # plt.title(f'Cosine Similarity Distribution for {model_name}')
    plt.xlabel('Cosine Similarity')
    plt.ylabel('Probability Density')

    plt.grid(True, linestyle="--", alpha=0.3)
    plt.tight_layout()

    plt.savefig(f"{model_name.replace(' ', '_')}_Cosine_Similarity.pdf", dpi=300, bbox_inches='tight', format="pdf")
    plt.show()


In [4]:

def get_qasm_vqe_final_loss(vqe_valid_file, n_steps):

    with h5py.File(vqe_valid_file, 'r') as f:
        random_loss_hist_group = f['loss_history']
        diff_loss_hist_group = f['diff_loss_history']
        q_loss_hist_group = f['dgnn_loss_history']
        ground_final_loss = []
        diff_final_loss = []
        q_final_loss = []
        rand_final_loss = []
        keys = list(random_loss_hist_group.keys())
        for key in keys: 
            random_loss_hist = random_loss_hist_group[key][...]
            diff_loss_hist = diff_loss_hist_group[key][...]
            q_loss_hist = q_loss_hist_group[key][...]
            ground_final_loss.append(random_loss_hist[-1][1])
            diff_final_loss.append(diff_loss_hist[n_steps][1])
            q_final_loss.append(q_loss_hist[n_steps][1])
            rand_final_loss.append(random_loss_hist[n_steps][1])

    return ground_final_loss, diff_final_loss, q_final_loss, rand_final_loss


def get_final_loss(valid_file, n_steps):

    with h5py.File(valid_file, 'r') as f:
        random_loss_hist_group = f['loss_history']
        diff_loss_hist_group = f['diff_loss_history']
        q_loss_hist_group = f['dgnn_loss_history']
        ground_final_loss = []
        diff_final_loss = []
        q_final_loss = []
        rand_final_loss = []
        keys = list(random_loss_hist_group.keys())
        for key in keys: 
            random_loss_hist = random_loss_hist_group[key][...]
            diff_loss_hist = diff_loss_hist_group[key][...]
            q_loss_hist = q_loss_hist_group[key][...]
            ground_final_loss.append(random_loss_hist[-1])
            diff_final_loss.append(diff_loss_hist[n_steps])
            q_final_loss.append(q_loss_hist[n_steps])
            rand_final_loss.append(random_loss_hist[n_steps])

    return ground_final_loss, diff_final_loss, q_final_loss, rand_final_loss


def compute_mre(y_label, y_pred):
    mask = y_label != 0
    return np.mean(np.abs((y_label[mask] - y_pred[mask]) / y_label[mask])) * 100  # MRE in percentage


def compute_smape(y_label, y_pred):
    mask = (y_label + y_pred) != 0
    return np.mean(np.abs(y_label[mask] - y_pred[mask]) / ((np.abs(y_label[mask]) + np.abs(y_pred[mask])) / 2)) * 100  # sMAPE in percentage

In [5]:

def plot_confusion_matrix(ground_energy, q_energy, model_name='QASMBench VQE', color = 'lightcoral', if_diffusion = False):
    mre = compute_mre(np.array(ground_energy), np.array(q_energy))


    min_val = min(np.min(ground_energy), np.min(q_energy))
    max_val = max(np.max(ground_energy), np.max(q_energy))

    x_line = np.linspace(min_val, max_val, 100)

    xtick = np.linspace(min_val, max_val, 5)
    ytick = np.linspace(min_val, max_val, 5)
    plt.figure()



    plt.scatter(ground_energy,q_energy, color=color, label = f'MRE = {mre: .2f}%')

    plt.plot(x_line, x_line, color='gray', linestyle='--')
    plt.legend()

    plt.xlabel(r'$E_{label}$')
    if if_diffusion:
        plt.ylabel(r'$E_{Diffusion}$')
    else:
        plt.ylabel(r'$E_{GNN}$')
    # plt.ylabel(r'$E_{Qracle}$')
    plt.xticks(xtick, labels=[f"{x:.2f}" for x in xtick])
    plt.yticks(ytick, labels=[f"{y:.2f}" for y in ytick])

    plt.grid(True, linestyle="--", alpha=0.3)
    plt.tight_layout()
    # plt.title(f'Confusion Matrix for {model_name}')

    plt.savefig(f"{model_name.replace(' ', '_')}_Confusion_Matrix.pdf", dpi=300, bbox_inches='tight', format="pdf")
    plt.show()



def get_init_losses(valid_file):
    """Reads the initial losses from the validation file."""
    random_init, diff_init, dgnn_init, gin_init = [], [], [], []

    # Read Data
    with h5py.File(valid_file, 'r') as f:
        for key in f['loss_history'].keys():
            random_init.append(f['loss_history'][key][0])
            diff_init.append(f['diff_loss_history'][key][0])
            dgnn_init.append(f['dgnn_loss_history'][key][0])
            gin_init.append(f['gin_loss_history'][key][0])

    return random_init, diff_init, dgnn_init, gin_init



def get_qasm_vqe_init_losses(valid_file):

    """Reads the initial losses from the QASM VQE validation file."""
    random_init, diff_init, dgnn_init, gin_init = [], [], [], []

    # Read Data
    with h5py.File(valid_file, 'r') as f:
        for key in f['loss_history'].keys():
            random_init.append(f['loss_history'][key][0, 1])
            diff_init.append(f['diff_loss_history'][key][0, 1])
            dgnn_init.append(f['dgnn_loss_history'][key][0, 1])
            gin_init.append(f['gin_loss_history'][key][0, 1])

    return random_init, diff_init, dgnn_init, gin_init




def get_qasm_conv_steps(valid_file, percentage_threshold=0.0001, plateau_steps=10):
    def get_steps_to_convergence(loss_history, percentage_threshold, plateau_steps):
        loss_values = loss_history[:, 1]
        total_height = max(loss_values) - min(loss_values)
        steps_to_convergence = len(loss_values)

        for i in range(len(loss_values)-plateau_steps):
            percentage_changes = [abs(loss_values[i+j+1] - loss_values[i+j])/total_height for j in range(plateau_steps)]
            if all (change < percentage_threshold for change in percentage_changes):
                steps_to_convergence = loss_history[i, 0]
                break

        return steps_to_convergence
    
    rand_conv_steps, diff_conv_steps, q_conv_steps = [], [], []

    with h5py.File(valid_file, 'r') as f:
        rand_loss_history_group = f['loss_history']
        diff_loss_history_group = f['diff_loss_history']
        q_loss_history_group = f['dgnn_loss_history']
 
        for key in rand_loss_history_group.keys():
            rand_loss_history = rand_loss_history_group[key][...]
            diff_loss_history = diff_loss_history_group[key][...]
            q_loss_history = q_loss_history_group[key][...]

            rand_steps_to_convergence = get_steps_to_convergence(rand_loss_history, percentage_threshold, plateau_steps)
            diff_steps_to_convergence = get_steps_to_convergence(diff_loss_history, percentage_threshold, plateau_steps)
            q_steps_to_convergence = get_steps_to_convergence(q_loss_history, percentage_threshold, plateau_steps)

            rand_conv_steps.append(rand_steps_to_convergence)
            diff_conv_steps.append(diff_steps_to_convergence)
            q_conv_steps.append(q_steps_to_convergence)
    return rand_conv_steps, diff_conv_steps, q_conv_steps
    

def get_conv_steps(valid_file, percentage_threshold=0.0001, plateau_steps=10):
    def get_steps_to_convergence(loss_history, percentage_threshold, plateau_steps):
        loss_values = loss_history
        total_height = max(loss_values) - min(loss_values)
        steps_to_convergence = len(loss_values)

        for i in range(len(loss_values)-plateau_steps):
            percentage_changes = [abs(loss_values[i+j+1] - loss_values[i+j])/total_height for j in range(plateau_steps)]
            if all (change < percentage_threshold for change in percentage_changes):
                steps_to_convergence = i + plateau_steps
                break

        return steps_to_convergence
    
    rand_conv_steps, diff_conv_steps, q_conv_steps = [], [], []

    with h5py.File(valid_file, 'r') as f:
        rand_loss_history_group = f['loss_history']
        diff_loss_history_group = f['diff_loss_history']
        q_loss_history_group = f['dgnn_loss_history']
 
        for key in rand_loss_history_group.keys():
            rand_loss_history = rand_loss_history_group[key][...]
            diff_loss_history = diff_loss_history_group[key][...]
            q_loss_history = q_loss_history_group[key][...]

            rand_steps_to_convergence = get_steps_to_convergence(rand_loss_history, percentage_threshold, plateau_steps)
            diff_steps_to_convergence = get_steps_to_convergence(diff_loss_history, percentage_threshold, plateau_steps)
            q_steps_to_convergence = get_steps_to_convergence(q_loss_history, percentage_threshold, plateau_steps)

            rand_conv_steps.append(rand_steps_to_convergence)
            diff_conv_steps.append(diff_steps_to_convergence)
            q_conv_steps.append(q_steps_to_convergence)
    return rand_conv_steps, diff_conv_steps, q_conv_steps
    

In [6]:
valid_files = [heisenberg_valid_file, ti_valid_file, fh_valid_file, molecule_valid_file]

model_names = ['Heisenberg', 'Transverse Ising', 'Fermi-Hubbard', 'H2']

n_steps = 200

for valid_file, model_name in zip(valid_files, model_names):
    ground_energy, diff_energy, q_energy, rand_energy = get_final_loss(valid_file, n_steps=n_steps)
    # plot_confusion_matrix(ground_energy, q_energy, model_name=model_name + 'GNN', color='lightcoral', if_diffusion=False)
    # plot_confusion_matrix(ground_energy, diff_energy, model_name=model_name + 'Diffusion', color='limegreen', if_diffusion=True)

    # plot_cosine_sim(valid_file, model_name=model_name)
    rand_mre = compute_mre(np.array(ground_energy), np.array(rand_energy))
    q_mre = compute_mre(np.array(ground_energy), np.array(q_energy))

    diff_mre = compute_mre(np.array(ground_energy), np.array(diff_energy))
    
    rand_smape = compute_smape(np.array(ground_energy), np.array(rand_energy))
    q_smape = compute_smape(np.array(ground_energy), np.array(q_energy))
    diff_smape = compute_smape(np.array(ground_energy), np.array(diff_energy))


    print(f"{model_name} Random sMAPE: {rand_smape:.2f}%, after {n_steps} steps")
    print(f"{model_name} Diffusion sMAPE: {diff_smape:.2f}%, after {n_steps} steps")
    print(f"{model_name} Qracle sMAPE: {q_smape:.2f}%, after {n_steps} steps")


    print(f"{model_name} Random MRE: {rand_mre:.2f}%, after {n_steps} steps")
    print(f"{model_name} Diffusion MRE: {diff_mre:.2f}%, after {n_steps} steps")
    print(f"{model_name} Qracle MRE: {q_mre:.2f}%, after {n_steps} steps")


Heisenberg Random sMAPE: 8.96%, after 200 steps
Heisenberg Diffusion sMAPE: 9.77%, after 200 steps
Heisenberg Qracle sMAPE: 4.09%, after 200 steps
Heisenberg Random MRE: 7.48%, after 200 steps
Heisenberg Diffusion MRE: 8.14%, after 200 steps
Heisenberg Qracle MRE: 3.52%, after 200 steps
Transverse Ising Random sMAPE: 2.84%, after 200 steps
Transverse Ising Diffusion sMAPE: 2.27%, after 200 steps
Transverse Ising Qracle sMAPE: 0.39%, after 200 steps
Transverse Ising Random MRE: 2.67%, after 200 steps
Transverse Ising Diffusion MRE: 1.96%, after 200 steps
Transverse Ising Qracle MRE: 0.37%, after 200 steps
Fermi-Hubbard Random sMAPE: 13.54%, after 200 steps
Fermi-Hubbard Diffusion sMAPE: 21.50%, after 200 steps
Fermi-Hubbard Qracle sMAPE: 15.07%, after 200 steps
Fermi-Hubbard Random MRE: 2133588677850728.50%, after 200 steps
Fermi-Hubbard Diffusion MRE: 1185068051145976.00%, after 200 steps
Fermi-Hubbard Qracle MRE: 1337957154529.60%, after 200 steps
H2 Random sMAPE: 2.01%, after 200 ste

In [7]:
ground_energy, diff_energy, q_energy, rand_energy= get_qasm_vqe_final_loss(vqe_valid_file, n_steps=n_steps)

qasm_q_mre = compute_mre(np.array(ground_energy), np.array(q_energy))
qasm_diff_mre = compute_mre(np.array(ground_energy), np.array(diff_energy))
qasm_rand_mre = compute_mre(np.array(ground_energy), np.array(rand_energy))
qasm_q_smape = compute_smape(np.array(ground_energy), np.array(q_energy))
qasm_diff_smape = compute_smape(np.array(ground_energy), np.array(diff_energy))
qasm_rand_smape = compute_smape(np.array(ground_energy), np.array(rand_energy))
print(f"QASM VQE Random sMAPE: {qasm_rand_smape:.2f}%, after {n_steps} steps")
print(f"QASM VQE Diffusion sMAPE: {qasm_diff_smape:.2f}%, after {n_steps} steps")
print(f"QASM VQE Qracle sMAPE: {qasm_q_smape:.2f}%, after {n_steps} steps")

print(f"QASM VQE Random MRE: {qasm_rand_mre:.2f}%, after {n_steps} steps")
print(f"QASM VQE Diffusion MRE: {qasm_diff_mre:.2f}%, after {n_steps} steps")
print(f"QASM VQE Qracle MRE: {qasm_q_mre:.2f}%, after {n_steps} steps")


QASM VQE Random sMAPE: 0.40%, after 200 steps
QASM VQE Diffusion sMAPE: 0.80%, after 200 steps
QASM VQE Qracle sMAPE: 0.40%, after 200 steps
QASM VQE Random MRE: 0.40%, after 200 steps
QASM VQE Diffusion MRE: 0.78%, after 200 steps
QASM VQE Qracle MRE: 0.40%, after 200 steps


In [8]:
for valid_file, model_name in zip(valid_files, model_names):
    random_init, diff_init, dgnn_init, _ = get_init_losses(valid_file)
    random_mean_init = np.mean(random_init)
    diff_mean_init = np.mean(diff_init)
    dgnn_mean_init = np.mean(dgnn_init)
    print(f"{model_name} Random Init Loss: {random_mean_init:.2f}")
    print(f"{model_name} Diff Init Loss: {diff_mean_init:.2f}")
    print(f"{model_name} Qracle Init Loss: {dgnn_mean_init:.2f}")

    # plot_confusion_matrix(random_init, diff_init, model_name=model_name + 'Diffusion', color='limegreen', if_diffusion=True)
    # plot_confusion_matrix(random_init, dgnn_init, model_name=model_name + 'GNN', color='lightcoral', if_diffusion=False)

Heisenberg Random Init Loss: 0.06
Heisenberg Diff Init Loss: -0.16
Heisenberg Qracle Init Loss: -3.11
Transverse Ising Random Init Loss: -3.23
Transverse Ising Diff Init Loss: -12.18
Transverse Ising Qracle Init Loss: -23.04
Fermi-Hubbard Random Init Loss: 1.01
Fermi-Hubbard Diff Init Loss: 0.47
Fermi-Hubbard Qracle Init Loss: -0.80
H2 Random Init Loss: -0.22
H2 Diff Init Loss: -0.36
H2 Qracle Init Loss: -0.58


In [9]:
random_init, diff_init, dgnn_init, _ = get_qasm_vqe_init_losses(vqe_valid_file)
random_mean_init = np.mean(random_init)
diff_mean_init = np.mean(diff_init)
dgnn_mean_init = np.mean(dgnn_init)
print(f"QASM VQE Random Init Loss: {random_mean_init:.2f}")
print(f"QASM VQE Diff Init Loss: {diff_mean_init:.2f}")
print(f"QASM VQE Qracle Init Loss: {dgnn_mean_init:.2f}")

QASM VQE Random Init Loss: 0.02
QASM VQE Diff Init Loss: 0.02
QASM VQE Qracle Init Loss: -0.08


In [10]:
rand_conv_steps, diff_conv_steps, q_conv_steps = get_qasm_conv_steps(vqe_valid_file)
print(f"QASM VQE Random Conv Steps: {np.mean(rand_conv_steps):.2f}")
print(f"QASM VQE Diffusion Conv Steps: {np.mean(diff_conv_steps):.2f}")
print(f"QASM VQE Qracle Conv Steps: {np.mean(q_conv_steps):.2f}")

/tmp/ipykernel_11028/799688225.py:80: RuntimeWarning: invalid value encountered in scalar divide
  percentage_changes = [abs(loss_values[i+j+1] - loss_values[i+j])/total_height for j in range(plateau_steps)]


QASM VQE Random Conv Steps: 237.82
QASM VQE Diffusion Conv Steps: 250.26
QASM VQE Qracle Conv Steps: 247.56


In [11]:
for valid_file, model_name in zip(valid_files, model_names):
    rand_conv_steps, diff_conv_steps, q_conv_steps = get_conv_steps(valid_file)
    print(f"{model_name} Random Conv Steps: {np.mean(rand_conv_steps):.2f}")
    print(f"{model_name} Diffusion Conv Steps: {np.mean(diff_conv_steps):.2f}")
    print(f"{model_name} Qracle Conv Steps: {np.mean(q_conv_steps):.2f}")

Heisenberg Random Conv Steps: 266.93
Heisenberg Diffusion Conv Steps: 251.89
Heisenberg Qracle Conv Steps: 187.47
Transverse Ising Random Conv Steps: 235.93
Transverse Ising Diffusion Conv Steps: 180.83
Transverse Ising Qracle Conv Steps: 158.50
Fermi-Hubbard Random Conv Steps: 218.46
Fermi-Hubbard Diffusion Conv Steps: 213.21
Fermi-Hubbard Qracle Conv Steps: 185.80
H2 Random Conv Steps: 215.91
H2 Diffusion Conv Steps: 600.00
H2 Qracle Conv Steps: 574.64
